In [90]:
import pandas as pd
import numpy as np

# pd.set_option('display.max_colwidth',1000)

In [91]:
ht_sponsored_df = pd.read_csv('./csv/sponsored/samples/hashtag_sponsored_2021-06-08 16_58_04.csv')
user_vids_sponsored_df = pd.read_csv('./csv/sponsored/samples/users_videos_sponsored.csv')
user_dets_sponsored_df = pd.read_csv('./csv/sponsored/samples/get_user_details_sponsored_2021-06-08 22_46_32.csv')

In [92]:
user_vids_sponsored_df['username'].value_counts().head(25)

paigemackenzie      2000
nba                 2000
kessyemely          2000
mrhamilton          2000
reitergrace         2000
katiefeeneyy        2000
lorengray           2000
chanydakota         2000
firstdateguide      1955
roryeliza           1944
jdrou               1916
manu                1898
themrspedersen      1891
kalaniballfree      1777
nita_shilimkar      1773
aliciamccarvell     1697
neetubisht          1667
mattheperson        1606
nmillz1             1600
itsa_meme_mario_    1512
vhackerr            1507
sohail_d            1454
tybottofficial      1409
sierra_zagarri      1392
xo.phiee            1373
Name: username, dtype: int64

In [93]:
user_dets_sponsored_df.head()

,user_name,user_id,nickname,account_created,verified,bio_link,followers,following,heart,heart_count,videos,diggs
0,soularty,6652451661401948166,Art for the Soul,1549079166,False,-,70700,149,1400000,1400000,162,0
1,ricardo.p1nto,6566533986121826310,Ricardo Pinto,1528862543,False,-,27400,633,187000,187000,209,0
2,dripnas,29014327,Naseer Johnson,1452600370,False,-,244600,743,4800000,4800000,146,0
3,miyaevarenae,6623756337799806982,Miyaeva Renae ♡,1542939687,False,https://beacons.page/miyaevarenae,4800000,164,142000000,142000000,475,0
4,linhbarbie,67563924000,✨ Linh Barbie ✨,1503955644,True,https://www.youtube.com/channel/UCDZ3_SK5RJgMm05TvE4Npww,16200000,224,262400000,262400000,1070,0


In [95]:
len(user_dets_sponsored_df)

194

In [96]:
len(user_vids_sponsored_df['username'].unique())

194

In [97]:
user_dets_sponsored_df.loc[user_dets_sponsored_df['user_name'] == 'nba']

,user_name,user_id,nickname,account_created,verified,bio_link,followers,following,heart,heart_count,videos,diggs
141,nba,134941634731003904,NBA,1472544108,True,https://app.link.nba.com/e/nbaschedulett,12700000,168,298000000,298000000,4348,0


In [104]:
user_dets_sponsored_df.loc[[126]]

,user_name,user_id,nickname,account_created,verified,bio_link,followers,following,heart,heart_count,videos,diggs
126,sisteryell,75398519921,옐언니,1510412985,True,http://bit.ly/sisteryellMD,10800000,24,237500000,237500000,895,4634


In [99]:
user_vids_sponsored_df.head()

,video_id,description,creation_time,duration,author_id,username,nickname,music_id,song_title,music_author_name,diggs,shares,comments,plays
0,6971501331164990725,WE SING A LOT HERE. BE MY BOYFRIEND FOR THE WEEK. YOUTUBE CHANNEL: CHRIS & IAN,1623179145,22,6747459471943418885,chris,Chris Olsen,6.971501e+18,original sound,Chris Olsen,7274,4,32,52400
1,6971222650995625221,HE ALWAYS TURNS IT AROUND @ THE END 😭 @ianpaget_,1623114261,41,6747459471943418885,chris,Chris Olsen,6.971223e+18,original sound,Chris Olsen,221900,1726,890,766700
2,6971214894595099910,HE GOT SO MAD AT RHE END 💀 @ianpaget_ #couple,1623112455,36,6747459471943418885,chris,Chris Olsen,6.971215e+18,original sound,Chris Olsen,254000,602,387,1300000
3,6970845224281066757,WHAT’D SHE SAYY AT THE ENDDD?? We love Lola ❤️🇵🇭,1623026384,33,6747459471943418885,chris,Chris Olsen,6.746993e+18,Monkeys Spinning Monkeys,Kevin MacLeod,328200,565,855,1400000
4,6970796148659391749,#stitch with @zaytashon88 Lola’s rockin w it 😤🇵🇭,1623014957,14,6747459471943418885,chris,Chris Olsen,6.970796e+18,original sound,Chris Olsen,1900000,7270,6067,8200000
